<a href="https://colab.research.google.com/github/Matrosee/Genre-Klassifikation-anhand-von-Liedtexten/blob/main/Code_von_Jannic_M_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installationen, Imports und Downloads:

Installationen:

In [ ]:
!pip install lightgbm
!pip install spacy
!pip install scikit-learn
!pip install numpy
!python -m spacy download en_core_web_md
!pip install scikit-learn
!pip install numpy
!pip install pandas
!pip install flaml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-01-23 12:21:14.228831: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 8.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
Looking in indexes:

Imports:

In [ ]:
import flaml
import lightgbm as lgb
import nltk
import numpy as np
import pandas as pd
import re
import spacy
from flaml import AutoML
from google.colab import files
from google.colab import drive
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Downloads:

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Upload & Dataframe Deklaration:

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/train.csv')

In [ ]:
df_test = pd.read_csv('/content/gdrive/MyDrive/test.csv')

# 1890 Songs-pro-Datensatz erstellen:

In [ ]:
# Nur Englische-Lieder einstellen
df = df.loc[df['Language']=='en']

# Pro Genre ein eigenes Dataframe mit je 1890 Liedern
df_rock_one = df[df['Genre']=='Rock']
df_rock_one = df_rock_one.head(1890)

df_pop_one = df[df['Genre']=='Pop']
df_pop_one = df_pop_one.head(1890)

df_metal_one = df[df['Genre']=='Metal']
df_metal_one = df_metal_one.head(1890)

df_hip_hop_one = df[df['Genre']=='Hip-Hop']
df_hip_hop_one = df_hip_hop_one.head(1890)

df_rnb_one = df[df['Genre']=='R&B']
df_rnb_one = df_rnb_one.head(1890)

df_indie_one = df[df['Genre']=='Indie']
df_indie_one = df_indie_one.head(1890)

df_electronic_one = df[df['Genre']=='Electronic']
df_electronic_one = df_electronic_one.head(1890)

df_jazz_one = df[df['Genre']=='Jazz']
df_jazz_one = df_jazz_one.head(1890)

df_folk_one = df[df['Genre']=='Folk']
df_folk_one = df_folk_one.head(1890)

df_country_one = df[df['Genre']=='Country']
df_country_one = df_country_one.head(1890)

# Alle einzelnen Dataframes zu einem zusammfügen
df_one = pd.concat([df_pop_one, df_hip_hop_one, df_metal_one, df_rock_one, 
                    df_indie_one, df_country_one, df_electronic_one, df_rnb_one,
                    df_jazz_one, df_folk_one])

# Kürzeren Namen für zuküftige Arbeit auswählen
df = df_one

# Spalten ohne Beiträge löschen 
df.dropna()

# Index reseten
df = df.reset_index(drop=True)

# Pre-Processing CountVectorizer & TF-IDF Vectorizer:

In [ ]:
df = df.dropna(axis=1)

In [ ]:
df = df.dropna(subset=['Lyrics'])

In [ ]:
# Laden Sie das spacy-Modell und den Englisch-Vokabular
nlp = spacy.load("en_core_web_md")

# Stop-Wörter auf einer Variable deklarieren
en_stops = set(stopwords.words('english'))

Datensatz säubern:

In [ ]:
# Sonderzeichen entfernen
df['Lyrics'] = [re.sub(r'^.*?Lyrics', '', str(lyric)) for lyric in df['Lyrics']]

# Satzumbruch Zeichen entfernen
df['cleaned_lyrics'] = [str(lyric).replace('\n',' ') for lyric in df['Lyrics']]

# Wörter in eckigen Klammern entfernen
df['cleaned_lyrics'] = [re.sub("\[.*?\]","",lyric) for lyric in df['cleaned_lyrics']]

# Language-, Song-Name- und Artist-Spalte löschen
df = df.drop('Language',axis=1)
df = df.drop('Song', axis=1)
df = df.drop('Artist', axis=1)

# Den Index resetten
df.reset_index(drop=True)

Stemming:

In [ ]:
# Neue Spalte erstellen und die Liedtexte ohne Endungen in die Spalte speichen
df['stemmed_lyrics'] = ""
stemmer = PorterStemmer() 
space = " "
tmp = ""
count = 0
for lyric in df['cleaned_lyrics']:
  words = word_tokenize(lyric)
  tmp = ""
  for word in words:
    tmp = tmp + space + stemmer.stem(word)
  df['stemmed_lyrics'].iloc[count] = tmp
  count = count + 1

Stop-Words:

In [ ]:
# Neue Spalte erstellen und die Liedtexte ohne Stoppwörter in die Spalte speichen
df['wosw_lyrics'] = ""
space = " "
tmp = ""
count = 0
for lyric in df['cleaned_lyrics']:
  words = word_tokenize(lyric)
  tmp = ""
  for word in words:
    if word not in en_stops:
      tmp = tmp + space + word
  df['wosw_lyrics'].iloc[count] = tmp
  count = count + 1

Stemming -> Stop-Words:

In [ ]:
# Neue Spalte erstellen und die Liedtexte ohne Endungen und ohne Stoppwörter in die Spalte speichen
df['stemmed_wosw_lyrics'] = ""
space = " "
tmp = ""
count = 0
for lyric in df['stemmed_lyrics']:
  words = word_tokenize(lyric)
  tmp = ""
  for word in words:
    if word not in en_stops:
      tmp = tmp + space + word
  df['stemmed_wosw_lyrics'].iloc[count] = tmp
  count = count + 1

StopWords -> Stemming:

In [ ]:
# Neue Spalte erstellen und die Liedtexte ohne Stoppwörter und ohne Endungen in die Spalte speichen
df['wosw_stemmed_lyrics'] = ""
ps = PorterStemmer() 
space = " "
tmp = ""
count = 0
for lyric in df['wosw_lyrics']:
  words = word_tokenize(lyric)
  tmp = ""
  for x in words:
    tmp = tmp + space + stemmer.stem(x)
  df['wosw_stemmed_lyrics'].iloc[count] = tmp
  count = count + 1

# Pre-Processing Spacy:

In [ ]:
# Laden Sie das spacy-Modell und den Englisch-Vokabular
nlp = spacy.load("en_core_web_md")

#stopwords rausfiltern
stop_words = set(stopwords.words("english"))

In [ ]:
# Laden Sie die Trainings-Textdokumente und die Trainings-Klassenlabels
text_train = df['Lyrics'].tolist() # Dataframe mit Textdokumenten
labels_train = df['Genre'] # Dataframe mit Klassenlabels (eins pro Dokument)

In [ ]:
# Laden Sie die Test-Textdokumente und die Test-Klassenlabels
text_test = df_test['Lyrics'].tolist() # Dataframe mit Textdokumenten
labels_test = df_test['Genre'] # Dataframe mit Klassenlabels (eins pro Dokument)

Trainingsdatensatz:

In [ ]:
#entfernen von Satzzeichen, Sonderzeichen und überflüssigen Wörtern
text_text = []
for t in text_train:
  t = t.lower()
  text_text.append(t)

In [ ]:
#entfernen von Satzzeichen, Sonderzeichen und überflüssigen Wörtern
cleaned_text = []
for t in text_train:
  t = t.lower()
  t = re.sub(r'[^\w\s]','',t)
  cleaned_text.append(t)

In [ ]:
stemmed_text = []
stemmer = PorterStemmer() 
for t in cleaned_text:
  stemmed_text.append(" ".join([stemmer.stem(word) for word in t.split()]))

In [ ]:
stopword_removed_text = []

for t in cleaned_text:
  t = t.lower()
  t = re.sub(r'[^\w\s]','',t)
  t = " ".join([word for word in t.split() if word not in stop_words])
  stopword_removed_text.append(t)

In [ ]:
stemmed_and_stopword_removed_text = []

for t in stemmed_text:
  t = " ".join([stemmer.stem(word) for word in t.split() if word not in stop_words])
  stemmed_and_stopword_removed_text.append(t)

In [ ]:
stopword_removed_and_stemmed_text = []

for t in stopword_removed_text:
  t = t.lower()
  t = re.sub(r'[^\w\s]','',t)
  t = " ".join([word for word in t.split() if word not in stop_words])
  t = " ".join([stemmer.stem(word) for word in t.split()])
  stopword_removed_and_stemmed_text.append(t)

Testdatensatz:

In [ ]:
#stopwords rausfiltern
stop_words = set(stopwords.words("english"))

In [ ]:
#entfernen von Satzzeichen, Sonderzeichen und überflüssigen Wörtern
text_text_test = []
for t in text_test:
  t = t.lower()
  text_text_test.append(t)

In [ ]:
#entfernen von Satzzeichen, Sonderzeichen und überflüssigen Wörtern
cleaned_text_test = []
for t in text_test:
  t = t.lower()
  t = re.sub(r'[^\w\s]','',t)
  cleaned_text_test.append(t)

In [ ]:
stemmed_text_test = []
stemmer = PorterStemmer() 
for t in cleaned_text_test:
  stemmed_text_test.append(" ".join([stemmer.stem(word) for word in t.split()]))

In [ ]:
stopword_removed_text_test = []

for t in cleaned_text_test:
  t = t.lower()
  t = re.sub(r'[^\w\s]','',t)
  t = " ".join([word for word in t.split() if word not in stop_words])
  stopword_removed_text_test.append(t)

In [ ]:
stemmed_and_stopword_removed_text_test = []

for t in stemmed_text_test:
  t = " ".join([stemmer.stem(word) for word in t.split() if word not in stop_words])
  stemmed_and_stopword_removed_text_test.append(t)

In [ ]:
stopword_removed_and_stemmed_text_test = []

for t in stopword_removed_text_test:
  t = t.lower()
  t = re.sub(r'[^\w\s]','',t)
  t = " ".join([word for word in t.split() if word not in stop_words])
  t = " ".join([stemmer.stem(word) for word in t.split()])
  stopword_removed_and_stemmed_text_test.append(t)

# Einstellen, welche Lyrik-Spalten verwendet werden sollen:

CountVectorizer & TF-IDF Vectorizer:

In [ ]:
# Laden Sie die Textdokumente und die Klassenlabels
df_train_lyrics = df['Lyrics'] # Dataframe mit Textdokumenten
df_train_labels = df['Genre'] # Dataframe mit Klassenlabels (eins pro Dokument)

In [ ]:
# Laden Sie die Textdokumente und die Klassenlabels
df_test_lyrics = df_test['Lyrics'] # Dataframe mit Textdokumenten
df_test_labels = df_test['Genre'] # Dataframe mit Klassenlabels (eins pro Dokument)

Spacy-Vectorizer:

In [ ]:
final_text_train = cleaned_text
final_text_test = cleaned_text_test

# Grid-search SVM:




CountVektorizer:

In [ ]:
# Konvertieren Sie die Textdokumente in numerische Merkmale mit dem CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df_train_lyrics)

# Teilen Sie die Daten in Trainings- und Testdaten auf
X_train, X_test, y_train, y_test = train_test_split(X, df_train_labels, test_size=0.2, random_state=42)

# Erstellen Sie ein SVM-Modell
svm = SVC()

# Definieren Sie die Hyperparameter, die Sie durchsuchen möchten
# Definieren Sie die Hyperparameter, die Sie durchsuchen möchten
param_grid_svm = {'C': [0.011, 0.012, 0.014], 
                  'gamma': [0.075, 0.079, 0.0795],
                  'kernel': ['linear']}

# Erstelle eine GridSearchCV-Instanz 
grid_svm = GridSearchCV(svm, param_grid_svm, refit = True, verbose = 3)

# Trainiere dein Modell mit der GridSearchCV
grid_svm.fit(X_train, y_train)

# Auswähle den besten Parameter
best_params_svm = grid_svm.best_params_

# Die besten Parameter ausgeben
print(best_params_svm)

TF-IDF-Vectorizer:

In [ ]:
# Konvertieren Sie die Textdokumente in numerische Merkmale mit dem TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_train_lyrics)

# Teilen Sie die Daten in Trainings- und Testdaten auf
X_train, X_test, y_train, y_test = train_test_split(X, df_train_labels, test_size=0.2, random_state=42)

# Erstellen Sie ein SVM-Modell
svm = SVC()

# Definieren Sie die Hyperparameter, die Sie durchsuchen möchten
# Definieren Sie die Hyperparameter, die Sie durchsuchen möchten
param_grid_svm = {'C': [4.5], 
                  'gamma': [0.95,1,3,5,10],
                  'kernel': ['rbf']}

# Erstelle eine GridSearchCV-Instanz 
grid_svm = GridSearchCV(svm, param_grid_svm, refit = True, verbose = 3)

# Trainiere dein Modell mit der GridSearchCV
grid_svm.fit(X_train, y_train)

# Auswähle den besten Parameter
best_params_svm = grid_svm.best_params_

# Die besten Parameter ausgeben
print(best_params_svm)

Spacy-Vectorizer:

In [ ]:
vectors = []

for t in text_train:
  doc = nlp(t)
  mean_vec = doc.vector
  vectors.append(mean_vec)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(vectors, labels_train, test_size=0.2, random_state=42)

# Erstellen Sie ein SVM-Modell
svm = SVC()

# Definieren Sie die Hyperparameter, die Sie durchsuchen möchten
# Definieren Sie die Hyperparameter, die Sie durchsuchen möchten
param_grid_svm = {'C': [1, 5, 10], 
                  'gamma': [1, 0.1, 0.001],
                  'kernel': ['linear']}

# Erstelle eine GridSearchCV-Instanz 
grid_svm = GridSearchCV(svm, param_grid_svm, refit = True, verbose = 3)

# Trainiere dein Modell mit der GridSearchCV
grid_svm.fit(X_train, y_train)

# Auswähle den besten Parameter
best_params_svm = grid_svm.best_params_

# Die besten Parameter ausgeben
print(best_params_svm)

# Flaml LGBM:

CountVectorizer:

In [ ]:
# Konvertieren Sie die Textdokumente in numerische Merkmale mit dem CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df_train_lyrics)

# Teilen Sie die Daten in Trainings- und Testdaten auf
X_train, X_test, y_train, y_test = train_test_split(X, df_train_labels, test_size=0.15, random_state=42)

# Das Flaml-Modell deklarieren und trainieren
automl = AutoML()
automl.fit(X_train, y_train, task="classification", metric='accuracy', estimator_list=["lgbm"], time_budget=3600)

# Die besten Hyperparameter ausgeben
print('Best hyperparmeter config:', automl.best_config)

TfidfVectorizer:

In [ ]:
# Konvertieren Sie die Textdokumente in numerische Merkmale mit dem CountVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_train_lyrics)

# Teilen Sie die Daten in Trainings- und Testdaten auf
X_train, X_test, y_train, y_test = train_test_split(X, df_train_labels, test_size=0.15, random_state=42)

# Das Flaml-Modell deklarieren und trainieren
automl = AutoML()
automl.fit(X_train, y_train, task="classification", metric='accuracy', estimator_list=["lgbm"], time_budget=6000)

# Die besten Hyperparameter ausgeben
print('Best hyperparmeter config:', automl.best_config)

Spacy-Vektoren:

In [ ]:
# Trainingsdatensatz in Spacy-Vektoren umwandeln
vectors = []

for t in text_train:
  doc = nlp(t)
  mean_vec = doc.vector
  vectors.append(mean_vec)

In [ ]:
# vectors in ein Numpy array konvertieren
vectors = np.array(vectors)

# Teilen Sie die Daten in Trainings- und Testdaten auf
X_train, X_test, y_train, y_test = train_test_split(vectors, labels_train, test_size=0.2, random_state=42)

# Das Flaml-Modell deklarieren und trainieren
automl = AutoML()
automl.fit(X_train, y_train, task="classification", metric='accuracy', estimator_list=["lgbm"], time_budget=4000)

# Die besten Hyperparameter ausgeben
print('Best hyperparmeter config:', automl.best_config)

# MLA:

CountVectorizer:

In [ ]:
# Vektorizer initialisiereen
vectorizer = CountVectorizer()

# Variablen für MLA deklarieren und teils konvertieren
X_train = vectorizer.fit_transform(df_train_lyrics)
X_test = vectorizer.transform(df_test_lyrics)

y_train = df_train_labels
y_test = df_test_labels

X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

In [ ]:
# Erstellen Sie das SVM- und LGBM-Modell
lgbm = lgb.LGBMClassifier(n_estimators=200, num_leaves= 184, min_child_samples=3, 
                          learning_rate=0.08838576141784195, log_max_bin=9, 
                          colsample_bytree=0.39039795455449644, 
                          reg_alpha=0.014844095616079196, 
                          reg_lambda=0.05525097930389173)
svm = SVC(C=0.01, gamma=0.07, kernel='linear')

# Trainieren Sie die Modelle anhand der Trainingsdaten
lgbm.fit(X_train, y_train)
svm.fit(X_train, y_train)

# Vorhersage der Testdaten svm
predictions_lgbm = lgbm.predict(X_test)
predictions_svm = svm.predict(X_test)

# Berechnen Sie die Genauigkeit
accuracy_lgbm = accuracy_score(y_test, predictions_lgbm)
accuracy_svm = accuracy_score(y_test, predictions_svm)

# Den Klassifikations-Report ausgeben
print('LGBM:')
print(classification_report(y_test, predictions_lgbm))
print('SVM:')
print(classification_report(y_test, predictions_svm))

# Die Genauigkeit ausgeben
print('Accuracy LGBM CouVec:', accuracy_lgbm)
print('Accuracy SVM CouVec:', accuracy_svm)

TF-IDF Vectorizer:

In [ ]:
# Vektorizer initialisiereen
vectorizer = TfidfVectorizer()

# Variablen für MLA deklarieren und teils konvertieren
X_train = vectorizer.fit_transform(df_train_lyrics)
X_test = vectorizer.transform(df_test_lyrics)

y_train = df_train_labels
y_test = df_test_labels

X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

In [ ]:
# Erstellen Sie das SVM- und LGBM-Modell
lgbm = lgb.LGBMClassifier(n_estimators=67, num_leaves= 184, min_child_samples=12, 
                          learning_rate=0.085537978248575, log_max_bin=9, 
                          colsample_bytree=0.7663773657187746, 
                          reg_alpha=0.006958608037974516, 
                          reg_lambda=0.4683303882185501)
svm = SVC(C=4.5, gamma=0.95, kernel='rbf')

# Trainieren Sie die Modelle anhand der Trainingsdaten
lgbm.fit(X_train, y_train)
svm.fit(X_train, y_train)

# Vorhersage der Testdaten svm
predictions_lgbm = lgbm.predict(X_test)
predictions_svm = svm.predict(X_test)

# Berechnen Sie die Genauigkeit
accuracy_lgbm = accuracy_score(y_test, predictions_lgbm)
accuracy_svm = accuracy_score(y_test, predictions_svm)

# Den Klassifikations-Report ausgeben
print('LGBM:')
print(classification_report(y_test, predictions_lgbm))
print('SVM:')
print(classification_report(y_test, predictions_svm))

# Die Genauigkeit ausgeben
print('Accuracy LGBM CouVec:', accuracy_lgbm)
print('Accuracy SVM CouVec:', accuracy_svm)

Spacy-Vectorizer:

In [ ]:
# Trainingsdatensatz in Spacy-Vektoren umwandeln
vectors_train = []

for t in final_text_train:
  doc = nlp(t)
  mean_vec = doc.vector
  vectors_train.append(mean_vec)

In [ ]:
# Testdatensatz in Spacy-Vektoren umwandeln
vectors_test = []

for t in final_text_test:
  doc = nlp(t)
  mean_vec = doc.vector
  vectors_test.append(mean_vec)

In [ ]:
# Variablen für MLA deklarieren
X_train = vectors_train
X_test = vectors_test

y_train = labels_train
y_test = labels_test

# Erstellen Sie das SVM- und LGBM-Modell
lgbm = lgb.LGBMClassifier()
svm = SVC(C=0.9, gamma=1.2, kernel='linear')

# Trainieren Sie die Modelle anhand der Trainingsdaten
lgbm.fit(X_train, y_train)
svm.fit(X_train, y_train)

# Berechnen Sie die Genauigkeit
accuracy_lgbm = lgbm.score(X_test, y_test)
accuracy_svm = svm.score(X_test, y_test)

# Genauigkeit printen
print("Accuracy LGBM: ", accuracy_lgbm)
print("Accuracy SVM: ", accuracy_svm)